In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import sys
sys.path.append("/project/")

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, PercentFormatter
import seaborn as sns
import matplotlib.lines as mlines


from utils import get_df

In [4]:
ALGO_LIST = [
    # "mcpg_me",
    # "dcg_me",
    # "pga_me",
    # "me",
    "memes",
    # "ppga",
]



In [6]:
results_dir = Path("memes_scal/output/")
EPISODE_LENGTH = 250
df = get_df(results_dir, EPISODE_LENGTH)
#df['algo'] = df.apply(filter, axis=1)
df = df[df["algo"].isin(ALGO_LIST)]
df = df[df["num_evaluations"] <= 1_005_000]

ant_omni_250
ant_uni_250
anttrap_omni_250
hopper_uni_250
walker2d_uni_250


In [7]:
idx = df.groupby(["env", "algo", "run"])["iteration"].idxmax()
df_last_iteration = df.loc[idx]
df = df_last_iteration[['env', 'algo', 'time', 'qd_score', 'batch_size']]

#df = df.groupby(['env', 'algo', 'batch_size'])[['qd_score', 'time']].median().reset_index()


df[['qd_score', 'time']] = df.groupby(['env', 'algo'])[['qd_score', 'time']].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

df = df.groupby(['env', 'algo', 'batch_size'])[['qd_score', 'time']].median().reset_index()

df['time'] = 1 - df['time']

df['scalability_score'] = df['qd_score'] * df['time']

#df['scalability_score'] = (0.5 * df['qd_score']) + (0.5 * df['time'])


df = df.groupby(['algo', 'batch_size'])[['scalability_score']].mean().reset_index()

/tmp/ipykernel_59131/2279556685.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['qd_score', 'time']] = df.groupby(['env', 'algo'])[['qd_score', 'time']].transform(


In [8]:
df

,algo,batch_size,scalability_score
0,memes,1,0.204171
1,memes,2,0.003686
2,memes,4,0.197038
3,memes,8,0.305737
4,memes,16,0.492796


In [6]:
df[df['algo']=='pga_me']

,env,algo,batch_size,qd_score,time
19,ant_omni_250,pga_me,256,0.000000,0.000000
20,ant_omni_250,pga_me,512,0.383819,0.600231
21,ant_omni_250,pga_me,1024,0.254231,0.880684
22,ant_omni_250,pga_me,2048,0.341926,0.944427
23,ant_omni_250,pga_me,4096,1.000000,1.000000
43,ant_uni_250,pga_me,256,1.000000,0.000000
44,ant_uni_250,pga_me,512,0.922535,0.603176
45,ant_uni_250,pga_me,1024,0.982445,0.886774
46,ant_uni_250,pga_me,2048,0.712709,0.949095
47,ant_uni_250,pga_me,4096,0.000000,1.000000


In [7]:
df = df.loc[df.groupby(['algo'])['scalability_score'].idxmax()]

In [8]:
df

,algo,batch_size,scalability_score
2,dcg_me,1024,0.686505
9,mcpg_me,4096,0.598087
14,me,4096,0.581361
17,memes,64,0.326350
21,pga_me,1024,0.458365
26,ppga,6000,0.305302
